In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [5]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Calculate the weights with different methods

In [6]:
from implementations import *

In [25]:
# Initialization
max_iters = 50
gamma = 0.2
lambda_=0.5
initial_w=np.zeros(30)

#Pour output un dossier .csv, assurez vous d'avoir train.csv et test.csv dans le même dossier que ce fichier
#Remplacer weightsi par weights si vous voulez utiliser la méthode i (i \in {1,2,3,4})
#Lancer les deux dernières cellues, le dossier en output aura le titre submission.csv
weights,_=least_squares_GD(y,tX,initial_w,max_iters,gamma)
#weights,_=least_squares_SGD(y,tX,initial_w,max_iters,gamma)
#weights,_=least_squares(y,tX)
#weights,_=ridge_regression(y,tX,lambda_)


In [26]:
print(weights)

[-5.12220179e+297  2.57704950e+297  3.88355865e+297  1.49490197e+297
 -4.82430314e+298 -4.90140651e+298 -4.82357220e+298  1.20759486e+296
  6.92349957e+296  4.72850920e+297  6.67759306e+295 -2.37473633e+295
 -4.82399074e+298  1.67767227e+297 -7.69603207e+293 -5.99728445e+293
  2.07366349e+297 -1.44094328e+294  2.05922743e+294  1.61975141e+297
 -8.42596941e+293  7.13844994e+297  1.60845526e+295 -2.88464038e+298
 -2.99087549e+298 -2.99090155e+298 -4.83064581e+298 -4.82392509e+298
 -4.82392603e+298  9.77173423e+296]


## Generate predictions and save ouput in csv format for submission:

In [14]:
DATA_TEST_PATH = 'test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [15]:
OUTPUT_PATH = 'submission.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)